In [18]:
import tensorflow as tf
from tensorflow.keras import layers, models

imagen_alto = 128
imagen_ancho = 128
canales_de_color = 1

# Definir la arquitectura de la CNN
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(imagen_alto, imagen_ancho, canales_de_color)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')  # 2 clases: Clase A y Clase B
]) 

# Compilar el modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])




c:\Users\achav\Documents\Osteoartritis\osteo\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
import os
import cv2
import numpy as np

# Directorio donde se encuentran las imágenes de entrenamiento
directorio_raiz = "dataset"

# Lista para almacenar las imágenes y etiquetas
imagenes = []
etiquetas = []

# Clases disponibles
clases = ['class_A', 'class_B']

# Recorrer las carpetas de las clases
for clase in clases:
    # Obtener la ruta completa de la carpeta de la clase actual
    ruta_clase = os.path.join(directorio_raiz, clase)
    # Unir con la ruta al tamaño de imagen deseado
    ruta_clase = os.path.join(ruta_clase, "size_128")

    # Obtener la etiqueta correspondiente a la clase actual
    etiqueta = clases.index(clase)
    # Recorrer las imágenes en la carpeta de la clase actual
    for imagen_nombre in os.listdir(ruta_clase):
        # Leer la imagen y convertirla a escala de grises
        imagen = cv2.imread(os.path.join(ruta_clase, imagen_nombre), cv2.IMREAD_GRAYSCALE)
        # Redimensionar la imagen si es necesario
        #imagen = cv2.resize(imagen, (nuevo_alto, nuevo_ancho))
        # Normalizar los valores de píxeles entre 0 y 1
        imagen = imagen / 255.0
        # Almacenar la imagen y su etiqueta en las listas correspondientes
        imagenes.append(imagen)
        etiquetas.append(etiqueta)

# Convertir las listas de imágenes y etiquetas a arrays numpy
imagenes = np.array(imagenes)
etiquetas = np.array(etiquetas)

num_epochs = 10
batch_size = 32

In [21]:
from sklearn.model_selection import train_test_split

# Dividir los datos en entrenamiento y validación
imagenes_entrenamiento, imagenes_validacion, etiquetas_entrenamiento, etiquetas_validacion = train_test_split(imagenes, etiquetas, test_size=0.2, random_state=42)

# Convertir las listas de imágenes y etiquetas de entrenamiento a arrays numpy
imagenes_entrenamiento = np.array(imagenes_entrenamiento)
etiquetas_entrenamiento = np.array(etiquetas_entrenamiento)

# Convertir las listas de imágenes y etiquetas de validación a arrays numpy
imagenes_validacion = np.array(imagenes_validacion)
etiquetas_validacion = np.array(etiquetas_validacion)

In [22]:
from tensorflow.keras.callbacks import TensorBoard

# Directorio donde se guardarán los registros para TensorBoard
directorio_logs = "logs/"

# Callback de TensorBoard
tensorboard_callback = TensorBoard(log_dir=directorio_logs, histogram_freq=1)

# Entrenar el modelo con el callback de TensorBoard
model.fit(imagenes_entrenamiento, etiquetas_entrenamiento,
          epochs=num_epochs,
          batch_size=batch_size,
          callbacks=[tensorboard_callback])




Epoch 1/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 8s 317ms/step - accuracy: 0.8380 - loss: 0.5575
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 319ms/step - accuracy: 0.8319 - loss: 0.4586
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 303ms/step - accuracy: 0.8011 - loss: 0.5288
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 278ms/step - accuracy: 0.8427 - loss: 0.4836
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 281ms/step - accuracy: 0.8231 - loss: 0.4670
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 267ms/step - accuracy: 0.8169 - loss: 0.4759
Epoch 7/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 273ms/step - accuracy: 0.8325 - loss: 0.4400
Epoch 8/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 273ms/step - accuracy: 0.8147 - loss: 0.4244
Epoch 9/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 279ms/step - accuracy: 0.8332 - loss: 0.3648
Epoch 10/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 273ms/step - accuracy: 0.8431 - loss: 0.3848


In [23]:
# Evaluar el modelo
test_loss, test_acc = model.evaluate(imagenes_validacion, etiquetas_validacion)
print('Precisión en datos de prueba:', test_acc)



5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 0.7662 - loss: 0.6852
Precisión en datos de prueba: 0.7659574747085571
